In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

# MultiIndex

In [3]:
idx_arr = [['a', 'a', 'b', 'b', 'c', 'c'], [1, 2, 1, 2, 1, 2]]
index = list(zip(*idx_arr))
index
multi_index = pd.MultiIndex.from_tuples(index, names=["first", "second"])
data = pd.Series(data=[1, 2, np.nan, 4, 5, 6], index=multi_index)
print(data.loc["b", 1:2])
print(data["b"][0:])

first  second
b      1         NaN
       2         4.0
dtype: float64
second
1    NaN
2    4.0
dtype: float64


In [4]:
data

first  second
a      1         1.0
       2         2.0
b      1         NaN
       2         4.0
c      1         5.0
       2         6.0
dtype: float64

In [5]:
ds = data.unstack() # to convert series to a dataframe, pivot table
print(type(ds))

<class 'pandas.core.frame.DataFrame'>


In [6]:
# multiindex on columns
data = pd.DataFrame(data=np.random.randint(0,10,size=16).reshape(4,4),
index = [["a", "a", "b", "b"], [1, 2, 1, 2, ]],
columns=[["ohio","ohio","or", "or"], ["green", "red", "green", "red"]])

In [7]:
data

ohio        or    
    green red green red
a 1     5   1     0   3
  2     4   3     0   3
b 1     8   3     6   1
  2     8   5     9   4

In [8]:
data.columns.names = ["state", "color"]
data.index.names = ["f", "s"]
data

state  ohio        or    
color green red green red
f s                      
a 1       5   1     0   3
  2       4   3     0   3
b 1       8   3     6   1
  2       8   5     9   4

In [9]:
data.swaplevel("f", "s")

state  ohio        or    
color green red green red
s f                      
1 a       5   1     0   3
2 a       4   3     0   3
1 b       8   3     6   1
2 b       8   5     9   4

In [10]:
data.swaplevel("state", "color", axis=1)

color green  red green red
state  ohio ohio    or  or
f s                       
a 1       5    1     0   3
  2       4    3     0   3
b 1       8    3     6   1
  2       8    5     9   4

In [11]:
data.sort_index(level=1)

state  ohio        or    
color green red green red
f s                      
a 1       5   1     0   3
b 1       8   3     6   1
a 2       4   3     0   3
b 2       8   5     9   4

In [12]:
data

state  ohio        or    
color green red green red
f s                      
a 1       5   1     0   3
  2       4   3     0   3
b 1       8   3     6   1
  2       8   5     9   4

In [13]:
data.reset_index() # makes named indexes as columns

state  f  s  ohio        or    
color       green red green red
0      a  1     5   1     0   3
1      a  2     4   3     0   3
2      b  1     8   3     6   1
3      b  2     8   5     9   4

In [14]:
data

state  ohio        or    
color green red green red
f s                      
a 1       5   1     0   3
  2       4   3     0   3
b 1       8   3     6   1
  2       8   5     9   4

In [15]:
data.groupby(level="f").sum()

state  ohio        or    
color green red green red
f                        
a         9   4     0   6
b        16   8    15   5

In [16]:
data.groupby("state", axis=1).sum()

state  ohio  or
f s            
a 1       6   3
  2       7   3
b 1      11   7
  2      13  13

In [17]:
np.random.seed(seed=42)
frame = pd.DataFrame({"a": range(7), "b": range(7, 0, -1),
                    "c": ["one", "one", "one", "two", "two",
                    "two", "two"],
                    "d": np.random.randint(1, 10, size=7,)})

frame

,a,b,c,d
0,0,7,one,7
1,1,6,one,4
2,2,5,one,8
3,3,4,two,5
4,4,3,two,7
5,5,2,two,3
6,6,1,two,7


In [18]:
frame1 = frame.set_index(["c", "d"]) # set columns as index

In [19]:
frame1

a  b
c   d      
one 7  0  7
    4  1  6
    8  2  5
two 5  3  4
    7  4  3
    3  5  2
    7  6  1

In [20]:
frame1 = frame1.sort_index(level="d")

In [21]:
frame1

a  b
c   d      
two 3  5  2
one 4  1  6
two 5  3  4
one 7  0  7
two 7  4  3
    7  6  1
one 8  2  5

In [22]:
frame1.index.levels

FrozenList([['one', 'two'], [3, 4, 5, 7, 8]])

In [23]:
frame1.index.nlevels

2

In [24]:
data = pd.Series(data=[1, 2, np.nan, 4, 5, 6], index=idx_arr)
data

data = pd.Series(data=[1, 2, np.nan, 4, 5, 6], index={'first':idx_arr[0], 'second': idx_arr[1]})
data

ValueError: Length of values (6) does not match length of index (2)

# Merge & Concat
- `merge` connects **rows** based on one or more keys
- `concat` stacks together objects along an axis
- `combine_first` splices overlapping data to fill in missing values
- `merge_asof` timeseries-based windows joins between DF objects

In [25]:
df1 = pd.DataFrame(
        {
           "A":  ["A0", "A1", "A2", "A3"],
           "B": ["B0", "B1", "B2", "B3"],
        "C": ["C0", "C1", "C2", "C3"],
            "D": ["D0", "D1", "D2", "D3"],
        },
        index=[0, 1, 2, 3],
    )
    

df2 = pd.DataFrame(
        {
            "A": ["A4", "A5", "A6", "A7"],
             "B": ["B4", "B5", "B6", "B7"],
             "C": ["C4", "C5", "C6", "C7"],
             "D": ["D4", "D5", "D6", "D7"],
        },
        index=[4, 5, 6, 7],
    )
    

df3 = pd.DataFrame(
        {
             "A": ["A8", "A9", "A10", "A11"],
             "B": ["B8", "B9", "B10", "B11"],
             "C": ["C8", "C9", "C10", "C11"],
             "D": ["D8", "D9", "D10", "D11"],
        },
        index=[8, 9, 10, 11],
    )
frames= [df1, df2, df3]

## Merge

In [26]:
df_merge = pd.merge(df1, df2, on=['A'], how="outer")
df_merge

,A,B_x,C_x,D_x,B_y,C_y,D_y
0,A0,B0,C0,D0,NaN,NaN,NaN
1,A1,B1,C1,D1,NaN,NaN,NaN
2,A2,B2,C2,D2,NaN,NaN,NaN
3,A3,B3,C3,D3,NaN,NaN,NaN
4,A4,NaN,NaN,NaN,B4,C4,D4
5,A5,NaN,NaN,NaN,B5,C5,D5
6,A6,NaN,NaN,NaN,B6,C6,D6
7,A7,NaN,NaN,NaN,B7,C7,D7


In [27]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "a", "b"],
"data1": pd.Series(range(7), dtype="Int64")})
df1 = df1.set_index("key")

df2 = pd.DataFrame({"key": ["a", "a", "b", "d"],
"data2": pd.Series(range(4), dtype="Int64")})
print(df1)
print(df2)

     data1
key       
b        0
b        1
a        2
c        3
a        4
a        5
b        6
  key  data2
0   a      0
1   a      1
2   b      2
3   d      3


In [28]:
pd.merge(df1, df2, on="key", how="right")

,key,data1,data2
0,a,2,0
1,a,4,0
2,a,5,0
3,a,2,1
4,a,4,1
5,a,5,1
6,b,0,2
7,b,1,2
8,b,6,2
9,d,<NA>,3


In [29]:
pd.merge(df1, df2, on="key", how="outer")

,key,data1,data2
0,b,0,2
1,b,1,2
2,b,6,2
3,a,2,0
4,a,2,1
5,a,4,0
6,a,4,1
7,a,5,0
8,a,5,1
9,c,3,<NA>


In [30]:
left = pd.DataFrame({"key1": ["foo", "foo", "bar"],
                         "key2": ["one", "two", "one"],
                         "val": pd.Series([1, 2, 3], dtype='Int64')})

right = pd.DataFrame({"key1": ["foo", "foo", "bar", "bar"],
                          "key2": ["one", "one", "one", "two"],
                          "val": pd.Series([4, 5, 6, 7], dtype='Int64')})
print(left)
print(right)
                    

  key1 key2  val
0  foo  one    1
1  foo  two    2
2  bar  one    3
  key1 key2  val
0  foo  one    4
1  foo  one    5
2  bar  one    6
3  bar  two    7


In [31]:
pd.merge(left, right, on=["key1", "key2"], sort=True, how="outer")                       

,key1,key2,val_x,val_y
0,bar,one,3,6
1,bar,two,<NA>,7
2,foo,one,1,4
3,foo,one,1,5
4,foo,two,2,<NA>


In [32]:
left = pd.DataFrame({"key1": ["foo", "foo", "bar"],
                         "key2": ["one", "two", "one"],
                         "val": pd.Series([1, 2, 3], dtype='Int64')})
left = left.set_index("key1")
                        

right = pd.DataFrame({"key1": ["foo", "foo", "bar", "bar"],
                          "key2": ["one", "one", "one", "two"],
                          "val": pd.Series([4, 5, 6, 7], dtype='Int64')})
print(left)
print(right)
                    

     key2  val
key1          
foo   one    1
foo   two    2
bar   one    3
  key1 key2  val
0  foo  one    4
1  foo  one    5
2  bar  one    6
3  bar  two    7


In [33]:
pd.merge(left, right, left_index=True, right_on="key1", sort=True, how="outer")

,key2_x,val_x,key1,key2_y,val_y
2,one,3,bar,one,6
3,one,3,bar,two,7
0,one,1,foo,one,4
1,one,1,foo,one,5
0,two,2,foo,one,4
1,two,2,foo,one,5


In [34]:
left1 = pd.DataFrame({"key": ["a", "b", "a", "a", "b", "c"],
                   "value": pd.Series(range(6), dtype="Int64")})

right1 = pd.DataFrame({"group_val": [3.5, 7]}, index=["a", "b"])
print(left1)
print(right1)

  key  value
0   a      0
1   b      1
2   a      2
3   a      3
4   b      4
5   c      5
   group_val
a        3.5
b        7.0


In [35]:
pd.merge(left1, right1, left_on="key", right_index=True, how="outer")

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


### Multi-index merging

In [36]:
lefth = pd.DataFrame({"key1": ["Ohio", "Ohio", "Ohio",
                                "Nevada", "Nevada"],
                       "key2": [2000, 2001, 2002, 2001, 2002],
                       "data": pd.Series(range(5), dtype="Int64")})

righth_index = pd.MultiIndex.from_arrays(
     [
         ["Nevada", "Nevada", "Ohio", "Ohio", "Ohio", "Ohio"],
         [2001, 2000, 2000, 2000, 2001, 2002]
     ]
 )

print(lefth)

     key1  key2  data
0    Ohio  2000     0
1    Ohio  2001     1
2    Ohio  2002     2
3  Nevada  2001     3
4  Nevada  2002     4


In [37]:
righth = pd.DataFrame({"event1": pd.Series([0, 2, 4, 6, 8, 10], dtype="Int64",
                                            index=righth_index),
                        "event2": pd.Series([1, 3, 5, 7, 9, 11], dtype="Int64",
                                            index=righth_index)})
print(righth)

             event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11


In [39]:
pd.merge(lefth, righth, left_on=["key1", "key2"], right_index=True, how="outer")

,key1,key2,data,event1,event2
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11
3,Nevada,2001,3,0,1
4,Nevada,2002,4,<NA>,<NA>
4,Nevada,2000,<NA>,2,3


In [41]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                      index=["a", "c", "e"],
                      columns=["Ohio", "Nevada"]).astype("Int64")

right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                       index=["b", "c", "d", "e"],
                       columns=["Missouri", "Alabama"]).astype("Int64")
print(left2)
print(right2)

   Ohio  Nevada
a     1       2
c     3       4
e     5       6
   Missouri  Alabama
b         7        8
c         9       10
d        11       12
e        13       14


In [43]:
pd.merge(left2, right2, left_index=True, right_index=True, how="outer")

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


### `join` to simplify merging by index

In [45]:
left2.join(right2, how="outer")

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


In [46]:
print(left1, "\n", right1)

  key  value
0   a      0
1   b      1
2   a      2
3   a      3
4   b      4
5   c      5 
    group_val
a        3.5
b        7.0


In [48]:
left1.join(right1, on="key", sort=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [55]:
right2

,Missouri,Alabama
b,7,8
c,9,10
d,11,12
e,13,14


In [58]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
   index=["a", "c", "e", "f"],
   columns=["New York", "Oregon"])

left3 = left1.set_index("key")
left3.join([right2, another], how="outer")
# left2.join([right2, another], how="outer")


,value,Missouri,Alabama,New York,Oregon
a,0,<NA>,<NA>,7.0,8.0
a,2,<NA>,<NA>,7.0,8.0
a,3,<NA>,<NA>,7.0,8.0
b,1,7,8,NaN,NaN
b,4,7,8,NaN,NaN
c,5,9,10,9.0,10.0
d,<NA>,11,12,NaN,NaN
e,<NA>,13,14,11.0,12.0
f,<NA>,<NA>,<NA>,16.0,17.0


## Concat
- by default in does `outer` join

### on numpy arrays

In [60]:
arr = np.arange(12).reshape((3, 4))
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

### on `Series`

In [61]:
s1 = pd.Series([0, 1], index=["a", "b"], dtype="Int64")

s2 = pd.Series([2, 3, 4], index=["c", "d", "e"], dtype="Int64")

s3 = pd.Series([5, 6], index=["f", "g"], dtype="Int64")

In [72]:
pd.concat([s1, s2, s3], axis="columns", keys=["s1", "s2", "s3"])
#pd.concat([s1, s2, s3], axis="index")

,s1,s2,s3
a,0,<NA>,<NA>
b,1,<NA>,<NA>
c,<NA>,2,<NA>
d,<NA>,3,<NA>
e,<NA>,4,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6


In [69]:
s4 = pd.concat([s1, s3])
s4
pd.concat([s1, s4], join="inner", axis=1)

a    0
b    1
f    5
g    6
dtype: Int64

In [78]:
result = pd.concat([s1, s2, s3], keys=["s1", "s2", "s3"])
result.unstack(level=1)
# result

,a,b,c,d,e,f,g
s1,0,1,<NA>,<NA>,<NA>,<NA>,<NA>
s2,<NA>,<NA>,2,3,4,<NA>,<NA>
s3,<NA>,<NA>,<NA>,<NA>,<NA>,5,6


### on `Dataframes`

In [80]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=["a", "b", "c"],
                   columns=["one", "two"])

df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=["a", "c"],
                   columns=["three", "four"])
print(df1, "\n", df2)

   one  two
a    0    1
b    2    3
c    4    5 
    three  four
a      5     6
c      7     8


In [81]:
pd.concat([df1, df2], axis=1, keys=["df1", "df2"])

df1       df2     
  one two three four
a   0   1   5.0  6.0
b   2   3   NaN  NaN
c   4   5   7.0  8.0

In [83]:
pd.concat({"df1": df1, "df2": df2}, axis=1)

df1       df2     
  one two three four
a   0   1   5.0  6.0
b   2   3   NaN  NaN
c   4   5   7.0  8.0

In [84]:

pd.concat({"df1": df1, "df2": df2}, axis=1, names=["source", "values"])

source df1       df2     
values one two three four
a        0   1   5.0  6.0
b        2   3   NaN  NaN
c        4   5   7.0  8.0

### `ignore_index`

In [85]:
df1 = pd.DataFrame(np.random.standard_normal((3, 4)),
                   columns=["a", "b", "c", "d"])


df2 = pd.DataFrame(np.random.standard_normal((2, 3)),
                   columns=["b", "d", "a"])

In [87]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,1.579213,0.767435,-0.469474,0.542560
1,-0.463418,-0.465730,0.241962,-1.913280
2,-1.724918,-0.562288,-1.012831,0.314247
3,1.465649,-0.908024,NaN,-1.412304
4,-1.424748,-0.225776,NaN,0.067528


## `combine_first` combine data with overlap

In [95]:
a = pd.Series([np.nan, 2.5, 0.0, 3.5, 4.5, np.nan],
              index=["f", "e", "d", "c", "b", "a"])
a = a.sort_index()

b = pd.Series([0., np.nan, 2., np.nan, np.nan, 5.],
              index=["a", "b", "c", "d", "e", "f"])

print(a, "\n", b)

a    NaN
b    4.5
c    3.5
d    0.0
e    2.5
f    NaN
dtype: float64 
 a    0.0
b    NaN
c    2.0
d    NaN
e    NaN
f    5.0
dtype: float64


In [96]:
np.where(a.isna(), b, a)

array([0. , 4.5, 3.5, 0. , 2.5, 5. ])

In [97]:
a.combine_first(b)

a    0.0
b    4.5
c    3.5
d    0.0
e    2.5
f    5.0
dtype: float64

In [99]:
df1 = pd.DataFrame({"a": [1., np.nan, 5., np.nan],
                    "b": [np.nan, 2., np.nan, 6.],
                    "c": range(2, 18, 4)})

df2 = pd.DataFrame({"a": [5., 4., np.nan, 3., 7.],
                    "b": [np.nan, 3., 4., 6., 8.]})
print(df1, "\n", df2)

     a    b   c
0  1.0  NaN   2
1  NaN  2.0   6
2  5.0  NaN  10
3  NaN  6.0  14 
      a    b
0  5.0  NaN
1  4.0  3.0
2  NaN  4.0
3  3.0  6.0
4  7.0  8.0


In [100]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


In [ ]:
df1 = pd.DataFrame(
        {
           "A":  ["A0", "A1", "A2", "A3"],
           "B": ["B0", "B1", "B2", "B3"],
        "C": ["C0", "C1", "C2", "C3"],
            "D": ["D0", "D1", "D2", "D3"],
        },
        index=[0, 1, 2, 3],
    )
    

df2 = pd.DataFrame(
        {
            "A": ["A4", "A5", "A6", "A7"],
             "B": ["B4", "B5", "B6", "B7"],
             "C": ["C4", "C5", "C6", "C7"],
             "D": ["D4", "D5", "D6", "D7"],
        },
        index=[4, 5, 6, 7],
    )
    

df3 = pd.DataFrame(
        {
             "A": ["A8", "A9", "A10", "A11"],
             "B": ["B8", "B9", "B10", "B11"],
             "C": ["C8", "C9", "C10", "C11"],
             "D": ["D8", "D9", "D10", "D11"],
        },
        index=[8, 9, 10, 11],
    )
frames= [df1, df2, df3]

In [124]:
pieces = {"x": df1, "y": df2, "z": df3}
r = pd.concat(pieces, levels=[["z", "x", "y", "w"]], keys=["x", "z"], names=["group_key"])
r

A    B    C    D
group_key                       
x         0    A0   B0   C0   D0
          1    A1   B1   C1   D1
          2    A2   B2   C2   D2
          3    A3   B3   C3   D3
z         8    A8   B8   C8   D8
          9    A9   B9   C9   D9
          10  A10  B10  C10  D10
          11  A11  B11  C11  D11

# Pivoting
- `stack` pivots from columns into rows
- `unstack` pivots from rows into columns

In [101]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
index=pd.Index(["Ohio", "Colorado"], name="state"),
columns=pd.Index(["one", "two", "three"],
name="number"))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [105]:
data.stack() # type is pd.Series

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [108]:
data.stack().unstack(level=0)
data.stack().unstack(level="state")


state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


- unstacking might introduce **missing data**

In [111]:
s1 = pd.Series([0, 1, 2, 3], index=["a", "b", "c", "d"], dtype="Int64")

s2 = pd.Series([4, 5, 6], index=["c", "d", "e"], dtype="Int64")
df = pd.concat([s1, s2], keys=["s1", "s2"])


In [112]:
df

s1  a    0
    b    1
    c    2
    d    3
s2  c    4
    d    5
    e    6
dtype: Int64

In [113]:
df.unstack(level=1)

,a,b,c,d,e
s1,0,1,2,3,<NA>
s2,<NA>,<NA>,4,5,6


- stacking filters out **missing data**

In [114]:
df.unstack(level=1).stack()

s1  a    0
    b    1
    c    2
    d    3
s2  c    4
    d    5
    e    6
dtype: Int64

In [115]:

df.unstack(level=1).stack(dropna=False)

s1  a       0
    b       1
    c       2
    d       3
    e    <NA>
s2  a    <NA>
    b    <NA>
    c       4
    d       5
    e       6
dtype: Int64

- unstacked level becomes the lowest level in the dataframe

In [117]:
result = data.stack()
df = pd.DataFrame({"left": result, "right": result + 5},
columns=pd.Index(["left", "right"], name="side"))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [118]:
df.unstack(level="state")

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [119]:

df.unstack(level="state").stack(level="side")

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

### long to wide

In [121]:
data = pd.read_csv("examples/macrodata.csv")
data

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.980,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.150,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.350,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.370,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.540,139.6,3.50,5.2,180.007,2.31,1.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,2008.0,3.0,13324.600,9267.7,1990.693,991.551,9838.3,216.889,1474.7,1.17,6.0,305.270,-3.16,4.33
199,2008.0,4.0,13141.920,9195.3,1857.661,1007.273,9920.4,212.174,1576.5,0.12,6.9,305.952,-8.79,8.91
200,2009.0,1.0,12925.410,9209.2,1558.494,996.287,9926.4,212.671,1592.8,0.22,8.1,306.547,0.94,-0.71
201,2009.0,2.0,12901.504,9189.0,1456.678,1023.528,10077.5,214.469,1653.6,0.18,9.2,307.226,3.37,-3.19


In [122]:
periods = pd.PeriodIndex(year=data.pop("year"),
quarter=data.pop("quarter"),
name="date")
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203)

In [124]:
data.index = periods.to_timestamp("D")
data.head()

,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
date,,,,,,,,,,,,
1959-01-01,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1959-04-01,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
1959-07-01,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
1959-10-01,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
1960-01-01,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [125]:
data.reindex(columns=["realgdp", "infl", "unemp"])

,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2
...,...,...,...
2008-07-01,13324.600,-3.16,6.0
2008-10-01,13141.920,-8.79,6.9
2009-01-01,12925.410,0.94,8.1


In [129]:
data = data[['realgdp', "infl", "unemp"]]
data.columns.name = "item"
data

item,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2
...,...,...,...
2008-07-01,13324.600,-3.16,6.0
2008-10-01,13141.920,-8.79,6.9
2009-01-01,12925.410,0.94,8.1


In [142]:
long_data = data.stack().reset_index().rename(columns={0: "value"})
long_data

,date,item,value
0,1959-01-01,realgdp,2710.349
1,1959-01-01,infl,0.000
2,1959-01-01,unemp,5.800
3,1959-04-01,realgdp,2778.801
4,1959-04-01,infl,2.340
...,...,...,...
604,2009-04-01,infl,3.370
605,2009-04-01,unemp,9.200
606,2009-07-01,realgdp,12990.341
607,2009-07-01,infl,3.560


- multiple time series data is frequently store in above long format in SQL
- fixd schema allows the unique values in `item` to chane as new data is added
- `date` and `item` would be primary keys offering both relational integrity and easier joins

- pivot tables make it easier to work with different item types

In [143]:
long_data.pivot(index="date", columns="item", values="value")

item,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2
...,...,...,...
2008-07-01,-3.16,13324.600,6.0
2008-10-01,-8.79,13141.920,6.9
2009-01-01,0.94,12925.410,8.1


 - pivoting with two value columns

In [147]:
long_data["value2"] = np.random.standard_normal((609,))
long_data

,date,item,value,value2
0,1959-01-01,realgdp,2710.349,-1.183259
1,1959-01-01,infl,0.000,-2.039232
2,1959-01-01,unemp,5.800,-0.269407
3,1959-04-01,realgdp,2778.801,0.717542
4,1959-04-01,infl,2.340,1.502357
...,...,...,...,...
604,2009-04-01,infl,3.370,-0.483186
605,2009-04-01,unemp,9.200,1.573987
606,2009-07-01,realgdp,12990.341,-1.225766
607,2009-07-01,infl,3.560,-1.464375


In [148]:
long_data.pivot(index="date", columns="item", values=["value", "value2"])

value                     value2                    
item        infl    realgdp unemp      infl   realgdp     unemp
date                                                           
1959-01-01  0.00   2710.349   5.8 -2.039232 -1.183259 -0.269407
1959-04-01  2.34   2778.801   5.1  1.502357  0.717542  0.074095
1959-07-01  2.74   2775.488   5.3 -1.380101  1.628616 -1.703382
1959-10-01  0.27   2785.204   5.6  0.384065 -0.055548 -0.032695
1960-01-01  2.31   2847.699   5.2 -0.089120 -2.067442 -1.304470
...          ...        ...   ...       ...       ...       ...
2008-07-01 -3.16  13324.600   6.0  0.436324  1.727543  0.038003
2008-10-01 -8.79  13141.920   6.9  0.613518  0.120031 -1.022793
2009-01-01  0.94  12925.410   8.1 -1.668584 -0.257377  0.399223
2009-04-01  3.37  12901.504   9.2 -0.483186  0.647196  1.573987
2009-07-01  3.56  12990.341   9.6 -1.464375 -1.225766  0.224452

[203 rows x 6 columns]

In [149]:
long_data

,date,item,value,value2
0,1959-01-01,realgdp,2710.349,-1.183259
1,1959-01-01,infl,0.000,-2.039232
2,1959-01-01,unemp,5.800,-0.269407
3,1959-04-01,realgdp,2778.801,0.717542
4,1959-04-01,infl,2.340,1.502357
...,...,...,...,...
604,2009-04-01,infl,3.370,-0.483186
605,2009-04-01,unemp,9.200,1.573987
606,2009-07-01,realgdp,12990.341,-1.225766
607,2009-07-01,infl,3.560,-1.464375


In [152]:
long_data.set_index(["date", "item"]).unstack(level="item")

value                     value2                    
item        infl    realgdp unemp      infl   realgdp     unemp
date                                                           
1959-01-01  0.00   2710.349   5.8 -2.039232 -1.183259 -0.269407
1959-04-01  2.34   2778.801   5.1  1.502357  0.717542  0.074095
1959-07-01  2.74   2775.488   5.3 -1.380101  1.628616 -1.703382
1959-10-01  0.27   2785.204   5.6  0.384065 -0.055548 -0.032695
1960-01-01  2.31   2847.699   5.2 -0.089120 -2.067442 -1.304470
...          ...        ...   ...       ...       ...       ...
2008-07-01 -3.16  13324.600   6.0  0.436324  1.727543  0.038003
2008-10-01 -8.79  13141.920   6.9  0.613518  0.120031 -1.022793
2009-01-01  0.94  12925.410   8.1 -1.668584 -0.257377  0.399223
2009-04-01  3.37  12901.504   9.2 -0.483186  0.647196  1.573987
2009-07-01  3.56  12990.341   9.6 -1.464375 -1.225766  0.224452

[203 rows x 6 columns]

### wide to long

#### `pd.melt`

In [153]:
df = pd.DataFrame({"key": ["foo", "bar", "baz"],
"A": [1, 2, 3],
"B": [4, 5, 6],
"C": [7, 8, 9]})

In [154]:
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [156]:
df.set_index("key").stack()

key   
foo  A    1
     B    4
     C    7
bar  A    2
     B    5
     C    8
baz  A    3
     B    6
     C    9
dtype: int64

In [161]:
melted = pd.melt(df, id_vars="key") # specify which columns are group indicators
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [164]:
melted.pivot(index="key", columns="variable", values="value").reset_index() # reset index to move key back into column

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [165]:
pd.melt(df, id_vars="key", value_vars=["A", "B"]) # subset of columns to stack

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [166]:
pd.melt(df, value_vars=["A", "B", "C"])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9
